In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from freedom.utils.dataset import Data
from freedom.utils.callback import Save

from freedom.neural_nets.hitnet import get_hitnet

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
data = Data(dirs=['/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols','/home/iwsatlas1/peller/work/oscNext/level3_v01.03/120000_i3cols'])

In [5]:
hitnet = get_hitnet(data.labels)
hitnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 9)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
hitnet_trafo_1 (hitnet_trafo)   (None, 17)           0           input_3[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 32)           576         hitnet_trafo_1[0][0]             
____________________________________________________________________________________________

In [5]:
optimizer = tf.keras.optimizers.Adam(0.00001)
hitnet.compile(loss='binary_crossentropy', optimizer=optimizer)

In [6]:
train_id = 'HitNet_logE' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [7]:
callbacks = []
callbacks.append(Save(save_every=2, path_template='models/'+train_id+'/epoch_%i'))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [8]:
train_data, test_data = data.get_hitnet_data(train_batch_size=4096,
                                             test_batch_size=4096,
                                             )

In [ ]:
hist = hitnet.fit(train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                  initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 1/1000
33572/33572 - 1267s - loss: 0.3367 - val_loss: 0.3163
Epoch 2/1000
33572/33572 - 1361s - loss: 0.3119 - val_loss: 0.3027
Epoch 3/1000
33572/33572 - 1171s - loss: 0.2993 - val_loss: 0.2899
Epoch 4/1000
33572/33572 - 1334s - loss: 0.2871 - val_loss: 0.2753
Epoch 5/1000


In [ ]:
# L7 0.28
# L3 0.239
# L3 with logE, angle correction, and pulse flags 0.205